In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%%capture
! pip install pyLDAvis
! pip install pymorphy2

In [3]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import gensim
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import json
import pymorphy2
import nltk
import re 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk import ngrams
import time
import dateutil.parser as dp
nltk.download('stopwords')
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime

russian_stopwords = stopwords.words("russian")
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
row_data = pd.read_json('drive/MyDrive/moretech/outputs.jsonl', lines = True, orient = 'records')

In [5]:
row_data

,link,text,title,source,timestamp,refers_to
0,https://www.garant.ru/news/1569973/,© ilixe48 / Фотобанк Фотодженика В этом матери...,Частичная мобилизация – 2022: трудовые права и...,garant,1665171180000,[https://www.garant.ru/ia/aggregator/?tag_id=2...
1,https://www.garant.ru/news/1578551/,© sonerbakir / Фотобанк Фотодженика Что измени...,Минприроды России подготовило разъяснения о по...,garant,1665170700000,"[https://base.garant.ru/77151921/, https://bas..."
2,https://www.garant.ru/news/1578549/,© hypnotype / Фотобанк Фотодженика Необходимо ...,Утверждены размеры платы за предоставление све...,garant,1665169920000,[https://premier.garantexpress.ru/premiergaran...
3,https://www.garant.ru/news/1578550/,© AndreyPopov / Фотобанк Фотодженика С 22 сент...,Утверждена формула для расчета платы за технол...,garant,1665169260000,[https://base.garant.ru/405378469/]
4,https://www.garant.ru/news/1578544/,"© ilze79 / Фотобанк Фотодженика АО ""Эксплуатир...",Президент РФ подписал указ об особенностях пра...,garant,1665167580000,[https://base.garant.ru/405385011/]
...,...,...,...,...,...,...
119223,https://www.banki.ru/news/lenta/?id=10963284,Уполномоченный при президенте России по защите...,Титов предложил ограничить ставку эквайринга п...,BANKIRU,1647945660000,[]
119224,https://www.banki.ru/news/lenta/?id=10963307,"Крупнейший в мире контейнерный перевозчик, дат...",Датская компания-перевозчик Maersk заявила о п...,BANKIRU,1647959100000,[]
119225,https://www.banki.ru/news/lenta/?id=10963286,Министерство финансов России предложило смягчи...,Минфин предложил втрое снизить штрафы за незак...,BANKIRU,1647949800000,[]
119226,https://www.banki.ru/news/lenta/?id=10963295,В России планируют создать единое облако для р...,Правительство предложит компаниям перейти на и...,BANKIRU,1647952380000,[https://www.kommersant.ru/doc/5270264?query=%...


In [6]:
class Preparator():
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.regexp = re.compile('[-%-,_\.\n\\\t\r/\W]')  # оставляет только буквы и цифры
        self.pos_decoder = {"ADJF": "ADJ",
                            'ADJS': 'ADJ',
                            'COMP': 'ADJ',
                            'INFN': 'VERB',
                            'PRTF': 'VERB'}

    def norm_word(self, word: str) -> str:
        w = str(re.sub(' +', ' ', self.regexp.sub(' ', word.lower()))).strip()
        if len(w) > 1 and w[0].isalpha():
            parsed = self.morph.parse(w)[0]
            tag = str(parsed.tag.POS)
            if tag not in ['NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']:
                tag = tag if tag not in self.pos_decoder else self.pos_decoder[tag]
                return parsed.normal_form.replace('ё', 'e')
        return ''
    
    def norm_sentence(self, sentence: str):
        tokens = list(filter(lambda x: len(x), [self.norm_word(word) for word in self.tokenizer.tokenize(sentence.lower())]))
        # bigrams = ['_'.join(w) for w in  ngrams(tokens,n=2)]
        return tokens
    
        

In [7]:
prep = Preparator()
texts = [prep.norm_sentence(i) for i in tqdm(row_data['title'])]

  0%|          | 0/119228 [00:00<?, ?it/s]

In [8]:
row_data.dtypes

link         object
text         object
title        object
source       object
timestamp     int64
refers_to    object
dtype: object

In [9]:
list(row_data.timestamp)[0]

1665171180000

In [10]:
min(row_data.timestamp), max(row_data.timestamp)

(-6795364578871, 1665181237000)

In [11]:
row_data['normed_text'] = texts
row_data['slice'] = [datetime.datetime.utcfromtimestamp(ts//1000).year for ts in row_data['timestamp']]
train, test = train_test_split(row_data, test_size=0.2, random_state=42, shuffle = True)

In [12]:
row_data.slice.value_counts()

2022    32376
2018    12735
2021    10610
2019    10352
2020     9473
2017     9171
2016     7527
2015     5419
2008     4302
2009     3412
2011     3378
2010     3134
2014     2570
2012     2188
2013     2041
2007      536
1754        4
Name: slice, dtype: int64

In [79]:
list(test.timestamp)[0]

1296739500000

In [80]:
dictionary = gensim.corpora.Dictionary(train.normed_text)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in train.normed_text]
time_slice = train.slice
num_topics = 20

In [ ]:
%%time
ldaseq = LdaSeqModel(corpus=doc_term_matrix, id2word=dictionary, time_slice=time_slice, num_topics=num_topics)
with open('ldaseq2.pkl', 'wb') as f:
     pickle.dump(ldaseq, f)